![terrainbento logo](../media/terrainbento_logo.png)


# terrainbento model Basic model with real DEM

This model shows example usage of the  [Basic](../coupled_process_elements/model_basic_steady_solution.ipynb)  model from the TerrainBento package. However, this time we will download and use an SRTM DEM as the initial condition. 

In [ ]:
# import required modules
import os

import holoviews as hv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from bmi_topography import Topography
from landlab.io import read_esri_ascii
from landlab.io.netcdf import write_netcdf
from landlab.plot.imshow import imshow_grid
from terrainbento import Basic

hv.notebook_extension("matplotlib")
# Ignore warnings
import warnings

import imageio

warnings.filterwarnings("ignore")

# Figure properties
%matplotlib inline
import panel as pn

pn.pane.Matplotlib.tight = True

## 1. Download SRTM image using the bmi-topography package
### bmi-topography
The bmi-topography package is a data component, recently developed by coding wizard Mark Piper. For more information on how to install an use it on your own machine, check out the [bmi-topography repo](https://github.com/csdms/bmi-topography) and this [notebook on bmi-topography](https://github.com/csdms/bmi-topography/blob/main/examples/topography.ipynb).

Bmi-topography is a Python library for fetching and caching NASA Shuttle Radar Topography Mission (SRTM) land elevation data using the [OpenTopography](https://opentopography.org/) [REST ](https://portal.opentopography.org/apidocs/) [API](https://www.mulesoft.com/resources/api/what-is-an-api).

The bmi-topography library provides access to the following global raster datasets:

    SRTM GL3 (90m)
    SRTM GL1 (30m)
    SRTM GL1 (Ellipsoidal)

The library includes an API and a CLI that accept the dataset type, a latitude-longitude bounding box, and the output file format. Data are downloaded from OpenTopography and cached locally. The cache is checked before downloading new data. 

The bmi-topography API is wrapped with a [Basic Model Interface (BMI)](https://bmi.readthedocs.io/), which provides a standard set of functions for coupling with data or models that also expose a BMI. More information on the BMI can found in its [documentation](https://bmi.readthedocs.io/).


### download some data

Create an instance of `Topography` using parameters to describe

* the type of data requested,
* the geographic bounding box of the data,
* the file format (we want to save it as an ascii file), and 
* where to store the file

with the following step:


In [ ]:
topo = Topography(
    dem_type="SRTMGL3",
    south=39.97,
    north=40.02,
    west=-105.32,
    east=-105.27,
    output_format="AAIGrid",
    cache_dir="DEMData//",
)

While this step sets up a call to the OpenTopography API, it doesn't download the data. Download the data by calling the `fetch` method :

In [ ]:
fname = topo.fetch()
print(fname)

This step may take a few moments to run while the data are fetched from OpenTopography and downloaded.

The `fetch` method only downloads data; it doesn't load it into memory. Call the `load` method to open the downloaded  file and load it into an `xarray` DataArray:

In [ ]:
dem = topo.load()
print(dem)

Note that `load` calls `fetch`, so the latter can be omitted if the goal is the get the data into memory.

## Visualize

Finally, let's visualize the downloaded elevation data.

In [ ]:
# Read DEM as Lanlab grid
grid, z = read_esri_ascii(fname, name="topographic__elevation")
# Show dem
plt.figure()
imshow_grid(
    grid,
    "topographic__elevation",
    cmap="terrain",
    grid_units=("deg", "deg"),
    var_name="Elevation (m)",
)
plt.show()

Yup, this is ![Boulder](../media/Boulder.jpg)

## 2. Use downloaded DEM to run a terrainbento model

In [ ]:
# Get properties of DEM
# Get properties of DEM
nr = grid.number_of_node_rows
nc = grid.number_of_node_columns
dx = grid.dx
xy_of_lower_left = grid.xy_of_lower_left

print(nr)
print(nc)
print(dx)
print(xy_of_lower_left)

In [ ]:
# create the parameter dictionary needed to instantiate the model

U = 0.001
K = 0.0005
m = 0.5
n = 1.0

params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 500,
        "stop": 1e4,
    },
    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (nr, nc),
            {
                "xy_spacing": dx,
                "xy_of_lower_left": xy_of_lower_left,
            },
            {
                "fields": {
                    "node": {"topographic__elevation": {"read_esri_ascii": [fname]}}
                }
            },
        ]
    },
    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {"modify_core_nodes": True, "lowering_rate": -U}
    },
    # Parameters that control output.
    "output_interval": 1e3,
    "save_first_timestep": True,
    "output_prefix": "output/RealDEM",
    "fields": ["topographic__elevation"],
    # Parameters that control process and rates.
    "water_erodibility": K,
    "m_sp": m,
    "n_sp": n,
    "regolith_transport_parameter": 1e-10,  # notice, this is degrees^2/year
}

In [ ]:
# initialize the model using the Model.from_dict() constructor.
# We also pass the output writer here.
flatIron_Festival = Basic.from_dict(params)

# to run the model as specified, we execute the following line:
flatIron_Festival.run()

In [ ]:
# make a plot of the final topography
plt.show()
plt.figure()
imshow_grid(
    flatIron_Festival.grid,
    "initial_topographic__elevation",
    cmap="terrain",
    grid_units=("deg", "deg"),
    var_name="Initial elevation (m)",
)
plt.show()

plt.figure()
imshow_grid(
    flatIron_Festival.grid,
    "topographic__elevation",
    cmap="terrain",
    grid_units=("deg", "deg"),
    var_name="flatIron_Festival (m)",
)

### Making a plot

terrainbento has a function called `to_xarray_dataset` that will take all the model output and combine it into one xarray datset. 
[More info on xarray datasets](http://xarray.pydata.org/en/stable/data-structures.html)

In [ ]:
ds = flatIron_Festival.to_xarray_dataset(time_unit="years", space_unit="deg")

To use Holoviews to make a plot we first create a holoviews dataset. 

In [ ]:
hvds_topo = hv.Dataset(ds.topographic__elevation)

Then we make a plot. Here we are will make an image plot with a time slider. Holoviews will render each image before showing the result so executing this cell may take a few moments. 

In [ ]:
topo = hvds_topo.to(hv.Image, ["x", "y"], label="flatIron_Festival").options(
    interpolation="bilinear", cmap="terrain", colorbar=True
)
topo.opts(
    fontsize={
        "title": 10,
        "labels": 10,
        "xticks": 10,
        "yticks": 10,
        "cticks": 10,
    }
)
topo

### Remove data

In [ ]:
flatIron_Festival.remove_output_netcdfs()

## Challenge: Run a Basic terrainbento model for the area where you are currently based

## Next Steps


- [Welcome page](../Welcome_to_TerrainBento.ipynb)


- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](../example_usage/Introduction_to_terrainbento.ipynb) 
    
    2) [Introduction to boundary conditions in terrainbento](../example_usage/introduction_to_boundary_conditions.ipynb)
    
    3) [Introduction to output writers in terrainbento](../example_usage/introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values
    
    6) **This Notebook**: [RealDEM](model_basic_realDEM.ipynb) Run the basic terrainbento model with a real DEM as initial condition. 